In [1]:
#참고링크 https://github.com/allenai/bilm-tf
#참고링크 https://appliedmachinelearning.wordpress.com/2019/11/30/training-elmo-from-scratch-on-custom-data-set-for-generating-embeddings-tensorflow/

In [1]:
#cpu 사용
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework

In [2]:
df_newsgroup = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/20news_dataset_clear/20newsgroup_preprocessed.csv', sep=';', usecols=['target', 'text_cleaned'])
df_newsgroup.rename(columns={'text_cleaned' : 'text'}, inplace=True)

In [3]:
le = LabelEncoder()
le.fit(df_newsgroup['target'].unique())

LabelEncoder()

In [4]:
df_newsgroup['target'] = le.transform(df_newsgroup['target'])

In [5]:
X = df_newsgroup['text'].astype(str)
# y = tf.keras.utils.to_categorical(df_newsgroup['target'], num_classes=df_newsgroup['target'].nunique())
y=list(df_newsgroup['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df_newsgroup['target'])

In [6]:
x_train=list(X_train)
x_test=list(X_test)

In [7]:
import re
pattern = '[^a-z.0-9 ]'
clr_x_trian=[]
clr_x_test=[]
for sen in x_train:
    clr_x_trian.append(re.sub(pattern, '', sen))
for sen in x_test:
    clr_x_test.append(re.sub(pattern, '', sen))

In [8]:
x_train=clr_x_trian
x_test=clr_x_test

In [9]:
to_txt_filter=x_train+x_test

In [10]:
y=list(y_train)+list(y_test)

엘모 벡터 생성

In [13]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers
 
# Location of pretrained LM.  Here we use the test fixtures.
datadir = os.path.join('swb', 'model(20ng 2만개 pro)')
vocab_file = os.path.join(datadir, 'vocab.txt')
options_file = os.path.join(datadir, 'options.json')
weight_file = os.path.join(datadir, 'swb_weights.hdf5')
 
# Create a Batcher to map text to character ids.
batcher = Batcher(vocab_file, 50)
 
# Input placeholders to the biLM.
context_character_ids = tf.placeholder('int32', shape=(None, None, 50))
 
# Build the biLM graph.
bilm = BidirectionalLanguageModel(options_file, weight_file)
 
# Get ops to compute the LM embeddings.
context_embeddings_op = bilm(context_character_ids)
 
# Get an op to compute ELMo (weighted average of the internal biLM layers)
elmo_context_input = weight_layers('input', context_embeddings_op, l2_coef=0.0)
 
# Now we can compute embeddings.
raw_context = ['Technology has advanced so much in new scientific world',
                'My child participated in fancy dress competition',
                'Fashion industry has seen tremendous growth in new designs']
 
tokenized_context = [sentence.split() for sentence in raw_context]
print(tokenized_context)

USING SKIP CONNECTIONS
[['Technology', 'has', 'advanced', 'so', 'much', 'in', 'new', 'scientific', 'world'], ['My', 'child', 'participated', 'in', 'fancy', 'dress', 'competition'], ['Fashion', 'industry', 'has', 'seen', 'tremendous', 'growth', 'in', 'new', 'designs']]


In [14]:
with tf.Session() as sess:
    # It is necessary to initialize variables once before running inference.
    sess.run(tf.global_variables_initializer())
 
    # Create batches of data.
    context_ids = batcher.batch_sentences(tokenized_context)
    print("Shape of context ids = ", context_ids.shape)
 
    # Compute ELMo representations (here for the input only, for simplicity).
    elmo_context_input_ = sess.run(
        elmo_context_input['weighted_op'],
        feed_dict={context_character_ids: context_ids}
    )
 
print("Shape of generated embeddings = ",elmo_context_input_.shape)

Shape of context ids =  (3, 11, 50)
Shape of generated embeddings =  (3, 9, 1024)


In [15]:
# Computing euclidean distance between words embedding
euc_dist_bet_tech_computer = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[0,0,:])
euc_dist_bet_computer_fashion = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[2,0,:])
# Computing cosine distance between words embedding
cos_dist_bet_tech_computer = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[0,0,:])
cos_dist_bet_computer_fashion = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[2,0,:])
 
print("Euclidean Distance Comparison - ")
print("\nDress-Technology = ",np.round(euc_dist_bet_tech_computer,2),"\nDress-Fashion = ",
      np.round(euc_dist_bet_computer_fashion,2))
print("\n\nCosine Distance Comparison - ")
print("\nDress-Technology = ",np.round(cos_dist_bet_tech_computer,2),"\nDress-Fashion = ",
      np.round(cos_dist_bet_computer_fashion,2))

Euclidean Distance Comparison - 

Dress-Technology =  49.59 
Dress-Fashion =  39.7


Cosine Distance Comparison - 

Dress-Technology =  1.03 
Dress-Fashion =  0.89


임베딩

In [16]:
except_lst=[]

In [17]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers

for i in range(len(x_train)):
    try:
        with tf.Session() as sess:

            sess.run(tf.global_variables_initializer())
            context_ids = batcher.batch_sentences([x_train[i].split()])
            elmo_context_input_ = sess.run(elmo_context_input['weighted_op'],feed_dict={context_character_ids: context_ids}) 
        print("Shape of generated embeddings = ",elmo_context_input_.shape)
        np.save('./embedding/train(20ng pro)/doc%d'%i,elmo_context_input_[0])
    except:
        except_lst.append(i)

Shape of generated embeddings =  (1, 83, 1024)
Shape of generated embeddings =  (1, 371, 1024)
Shape of generated embeddings =  (1, 730, 1024)
Shape of generated embeddings =  (1, 106, 1024)
Shape of generated embeddings =  (1, 80, 1024)
Shape of generated embeddings =  (1, 80, 1024)
Shape of generated embeddings =  (1, 113, 1024)
Shape of generated embeddings =  (1, 116, 1024)
Shape of generated embeddings =  (1, 33, 1024)
Shape of generated embeddings =  (1, 94, 1024)
Shape of generated embeddings =  (1, 22, 1024)
Shape of generated embeddings =  (1, 150, 1024)
Shape of generated embeddings =  (1, 28, 1024)
Shape of generated embeddings =  (1, 133, 1024)
Shape of generated embeddings =  (1, 69, 1024)
Shape of generated embeddings =  (1, 70, 1024)
Shape of generated embeddings =  (1, 99, 1024)
Shape of generated embeddings =  (1, 60, 1024)
Shape of generated embeddings =  (1, 13, 1024)
Shape of generated embeddings =  (1, 55, 1024)
Shape of generated embeddings =  (1, 43, 1024)
Shape 

Shape of generated embeddings =  (1, 53, 1024)
Shape of generated embeddings =  (1, 123, 1024)
Shape of generated embeddings =  (1, 33, 1024)
Shape of generated embeddings =  (1, 29, 1024)
Shape of generated embeddings =  (1, 43, 1024)
Shape of generated embeddings =  (1, 177, 1024)
Shape of generated embeddings =  (1, 86, 1024)
Shape of generated embeddings =  (1, 51, 1024)
Shape of generated embeddings =  (1, 44, 1024)
Shape of generated embeddings =  (1, 424, 1024)
Shape of generated embeddings =  (1, 79, 1024)
Shape of generated embeddings =  (1, 254, 1024)
Shape of generated embeddings =  (1, 94, 1024)
Shape of generated embeddings =  (1, 263, 1024)
Shape of generated embeddings =  (1, 20, 1024)
Shape of generated embeddings =  (1, 37, 1024)
Shape of generated embeddings =  (1, 58, 1024)
Shape of generated embeddings =  (1, 86, 1024)
Shape of generated embeddings =  (1, 108, 1024)
Shape of generated embeddings =  (1, 42, 1024)
Shape of generated embeddings =  (1, 65, 1024)
Shape o

Shape of generated embeddings =  (1, 111, 1024)
Shape of generated embeddings =  (1, 2200, 1024)
Shape of generated embeddings =  (1, 105, 1024)
Shape of generated embeddings =  (1, 80, 1024)
Shape of generated embeddings =  (1, 146, 1024)
Shape of generated embeddings =  (1, 57, 1024)
Shape of generated embeddings =  (1, 129, 1024)
Shape of generated embeddings =  (1, 120, 1024)
Shape of generated embeddings =  (1, 66, 1024)
Shape of generated embeddings =  (1, 58, 1024)
Shape of generated embeddings =  (1, 56, 1024)
Shape of generated embeddings =  (1, 113, 1024)
Shape of generated embeddings =  (1, 43, 1024)
Shape of generated embeddings =  (1, 117, 1024)
Shape of generated embeddings =  (1, 20, 1024)
Shape of generated embeddings =  (1, 197, 1024)
Shape of generated embeddings =  (1, 722, 1024)
Shape of generated embeddings =  (1, 119, 1024)
Shape of generated embeddings =  (1, 31, 1024)
Shape of generated embeddings =  (1, 39, 1024)
Shape of generated embeddings =  (1, 77, 1024)
S

Shape of generated embeddings =  (1, 34, 1024)
Shape of generated embeddings =  (1, 138, 1024)
Shape of generated embeddings =  (1, 67, 1024)
Shape of generated embeddings =  (1, 67, 1024)
Shape of generated embeddings =  (1, 11, 1024)
Shape of generated embeddings =  (1, 90, 1024)
Shape of generated embeddings =  (1, 47, 1024)
Shape of generated embeddings =  (1, 37, 1024)
Shape of generated embeddings =  (1, 74, 1024)
Shape of generated embeddings =  (1, 115, 1024)
Shape of generated embeddings =  (1, 28, 1024)
Shape of generated embeddings =  (1, 161, 1024)
Shape of generated embeddings =  (1, 68, 1024)
Shape of generated embeddings =  (1, 104, 1024)
Shape of generated embeddings =  (1, 44, 1024)
Shape of generated embeddings =  (1, 350, 1024)
Shape of generated embeddings =  (1, 128, 1024)
Shape of generated embeddings =  (1, 139, 1024)
Shape of generated embeddings =  (1, 36, 1024)
Shape of generated embeddings =  (1, 169, 1024)
Shape of generated embeddings =  (1, 70, 1024)
Shape

Shape of generated embeddings =  (1, 56, 1024)
Shape of generated embeddings =  (1, 144, 1024)
Shape of generated embeddings =  (1, 282, 1024)
Shape of generated embeddings =  (1, 27, 1024)
Shape of generated embeddings =  (1, 149, 1024)
Shape of generated embeddings =  (1, 149, 1024)
Shape of generated embeddings =  (1, 80, 1024)
Shape of generated embeddings =  (1, 147, 1024)
Shape of generated embeddings =  (1, 3050, 1024)
Shape of generated embeddings =  (1, 37, 1024)
Shape of generated embeddings =  (1, 22, 1024)
Shape of generated embeddings =  (1, 91, 1024)
Shape of generated embeddings =  (1, 101, 1024)
Shape of generated embeddings =  (1, 54, 1024)
Shape of generated embeddings =  (1, 119, 1024)
Shape of generated embeddings =  (1, 136, 1024)
Shape of generated embeddings =  (1, 359, 1024)
Shape of generated embeddings =  (1, 106, 1024)
Shape of generated embeddings =  (1, 89, 1024)
Shape of generated embeddings =  (1, 54, 1024)
Shape of generated embeddings =  (1, 59, 1024)
S

...

Shape of generated embeddings =  (1, 232, 1024)
Shape of generated embeddings =  (1, 529, 1024)
Shape of generated embeddings =  (1, 76, 1024)
Shape of generated embeddings =  (1, 107, 1024)
Shape of generated embeddings =  (1, 214, 1024)
Shape of generated embeddings =  (1, 169, 1024)
Shape of generated embeddings =  (1, 81, 1024)
Shape of generated embeddings =  (1, 37, 1024)
Shape of generated embeddings =  (1, 251, 1024)
Shape of generated embeddings =  (1, 163, 1024)
Shape of generated embeddings =  (1, 69, 1024)
Shape of generated embeddings =  (1, 40, 1024)
Shape of generated embeddings =  (1, 96, 1024)
Shape of generated embeddings =  (1, 80, 1024)
Shape of generated embeddings =  (1, 56, 1024)
Shape of generated embeddings =  (1, 60, 1024)
Shape of generated embeddings =  (1, 96, 1024)
Shape of generated embeddings =  (1, 96, 1024)
Shape of generated embeddings =  (1, 146, 1024)
Shape of generated embeddings =  (1, 89, 1024)
Shape of generated embeddings =  (1, 1005, 1024)
Sha

Shape of generated embeddings =  (1, 44, 1024)
Shape of generated embeddings =  (1, 74, 1024)
Shape of generated embeddings =  (1, 134, 1024)
Shape of generated embeddings =  (1, 248, 1024)
Shape of generated embeddings =  (1, 304, 1024)
Shape of generated embeddings =  (1, 65, 1024)
Shape of generated embeddings =  (1, 157, 1024)
Shape of generated embeddings =  (1, 18, 1024)
Shape of generated embeddings =  (1, 242, 1024)
Shape of generated embeddings =  (1, 177, 1024)
Shape of generated embeddings =  (1, 47, 1024)
Shape of generated embeddings =  (1, 19, 1024)
Shape of generated embeddings =  (1, 232, 1024)
Shape of generated embeddings =  (1, 464, 1024)
Shape of generated embeddings =  (1, 21, 1024)
Shape of generated embeddings =  (1, 71, 1024)
Shape of generated embeddings =  (1, 87, 1024)
Shape of generated embeddings =  (1, 161, 1024)
Shape of generated embeddings =  (1, 83, 1024)
Shape of generated embeddings =  (1, 73, 1024)
Shape of generated embeddings =  (1, 52, 1024)
Shap

Shape of generated embeddings =  (1, 64, 1024)
Shape of generated embeddings =  (1, 187, 1024)
Shape of generated embeddings =  (1, 375, 1024)
Shape of generated embeddings =  (1, 396, 1024)
Shape of generated embeddings =  (1, 994, 1024)
Shape of generated embeddings =  (1, 18, 1024)
Shape of generated embeddings =  (1, 116, 1024)
Shape of generated embeddings =  (1, 31, 1024)
Shape of generated embeddings =  (1, 11, 1024)
Shape of generated embeddings =  (1, 43, 1024)
Shape of generated embeddings =  (1, 45, 1024)
Shape of generated embeddings =  (1, 383, 1024)
Shape of generated embeddings =  (1, 74, 1024)
Shape of generated embeddings =  (1, 80, 1024)
Shape of generated embeddings =  (1, 215, 1024)
Shape of generated embeddings =  (1, 120, 1024)
Shape of generated embeddings =  (1, 48, 1024)
Shape of generated embeddings =  (1, 40, 1024)
Shape of generated embeddings =  (1, 44, 1024)
Shape of generated embeddings =  (1, 15, 1024)
Shape of generated embeddings =  (1, 535, 1024)
Shap

Shape of generated embeddings =  (1, 51, 1024)
Shape of generated embeddings =  (1, 87, 1024)
Shape of generated embeddings =  (1, 135, 1024)
Shape of generated embeddings =  (1, 103, 1024)
Shape of generated embeddings =  (1, 65, 1024)
Shape of generated embeddings =  (1, 81, 1024)
Shape of generated embeddings =  (1, 24, 1024)
Shape of generated embeddings =  (1, 60, 1024)
Shape of generated embeddings =  (1, 23, 1024)
Shape of generated embeddings =  (1, 29, 1024)
Shape of generated embeddings =  (1, 234, 1024)
Shape of generated embeddings =  (1, 74, 1024)
Shape of generated embeddings =  (1, 42, 1024)
Shape of generated embeddings =  (1, 115, 1024)
Shape of generated embeddings =  (1, 26, 1024)
Shape of generated embeddings =  (1, 109, 1024)
Shape of generated embeddings =  (1, 65, 1024)
Shape of generated embeddings =  (1, 100, 1024)
Shape of generated embeddings =  (1, 236, 1024)
Shape of generated embeddings =  (1, 246, 1024)
Shape of generated embeddings =  (1, 217, 1024)
Shap

Shape of generated embeddings =  (1, 1139, 1024)
Shape of generated embeddings =  (1, 45, 1024)
Shape of generated embeddings =  (1, 3, 1024)
Shape of generated embeddings =  (1, 102, 1024)
Shape of generated embeddings =  (1, 40, 1024)
Shape of generated embeddings =  (1, 70, 1024)
Shape of generated embeddings =  (1, 34, 1024)
Shape of generated embeddings =  (1, 80, 1024)
Shape of generated embeddings =  (1, 68, 1024)
Shape of generated embeddings =  (1, 36, 1024)
Shape of generated embeddings =  (1, 92, 1024)
Shape of generated embeddings =  (1, 26, 1024)
Shape of generated embeddings =  (1, 48, 1024)
Shape of generated embeddings =  (1, 497, 1024)
Shape of generated embeddings =  (1, 45, 1024)
Shape of generated embeddings =  (1, 261, 1024)
Shape of generated embeddings =  (1, 68, 1024)
Shape of generated embeddings =  (1, 139, 1024)
Shape of generated embeddings =  (1, 108, 1024)
Shape of generated embeddings =  (1, 73, 1024)
Shape of generated embeddings =  (1, 115, 1024)
Shape 

In [18]:
except_lst

[12935]

In [19]:
except_test_lst=[]

In [20]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers

for i in range(len(x_test)):
    try:
        with tf.Session() as sess:

            sess.run(tf.global_variables_initializer())
            context_ids = batcher.batch_sentences([x_test[i].split()])
            elmo_context_input_ = sess.run(elmo_context_input['weighted_op'],feed_dict={context_character_ids: context_ids}) 
        print("Shape of generated embeddings = ",elmo_context_input_.shape)
        np.save('./embedding/test(20ng pro)/test%d'%i,elmo_context_input_[0])
    except:
        except_test_lst.append(i)

Shape of generated embeddings =  (1, 21, 1024)
Shape of generated embeddings =  (1, 65, 1024)
Shape of generated embeddings =  (1, 90, 1024)
Shape of generated embeddings =  (1, 107, 1024)
Shape of generated embeddings =  (1, 107, 1024)
Shape of generated embeddings =  (1, 98, 1024)
Shape of generated embeddings =  (1, 99, 1024)
Shape of generated embeddings =  (1, 248, 1024)
Shape of generated embeddings =  (1, 83, 1024)
Shape of generated embeddings =  (1, 20, 1024)
Shape of generated embeddings =  (1, 33, 1024)
Shape of generated embeddings =  (1, 22, 1024)
Shape of generated embeddings =  (1, 45, 1024)
Shape of generated embeddings =  (1, 39, 1024)
Shape of generated embeddings =  (1, 107, 1024)
Shape of generated embeddings =  (1, 92, 1024)
Shape of generated embeddings =  (1, 67, 1024)
Shape of generated embeddings =  (1, 104, 1024)
Shape of generated embeddings =  (1, 99, 1024)
Shape of generated embeddings =  (1, 75, 1024)
Shape of generated embeddings =  (1, 79, 1024)
Shape of

Shape of generated embeddings =  (1, 53, 1024)
Shape of generated embeddings =  (1, 79, 1024)
Shape of generated embeddings =  (1, 129, 1024)
Shape of generated embeddings =  (1, 94, 1024)
Shape of generated embeddings =  (1, 21, 1024)
Shape of generated embeddings =  (1, 174, 1024)
Shape of generated embeddings =  (1, 208, 1024)
Shape of generated embeddings =  (1, 58, 1024)
Shape of generated embeddings =  (1, 26, 1024)
Shape of generated embeddings =  (1, 45, 1024)
Shape of generated embeddings =  (1, 160, 1024)
Shape of generated embeddings =  (1, 30, 1024)
Shape of generated embeddings =  (1, 78, 1024)
Shape of generated embeddings =  (1, 93, 1024)
Shape of generated embeddings =  (1, 156, 1024)
Shape of generated embeddings =  (1, 464, 1024)
Shape of generated embeddings =  (1, 177, 1024)
Shape of generated embeddings =  (1, 307, 1024)
Shape of generated embeddings =  (1, 61, 1024)
Shape of generated embeddings =  (1, 5005, 1024)
Shape of generated embeddings =  (1, 16, 1024)
Sha

Shape of generated embeddings =  (1, 139, 1024)
Shape of generated embeddings =  (1, 62, 1024)
Shape of generated embeddings =  (1, 32, 1024)
Shape of generated embeddings =  (1, 85, 1024)
Shape of generated embeddings =  (1, 63, 1024)
Shape of generated embeddings =  (1, 37, 1024)
Shape of generated embeddings =  (1, 60, 1024)
Shape of generated embeddings =  (1, 108, 1024)
Shape of generated embeddings =  (1, 31, 1024)
Shape of generated embeddings =  (1, 25, 1024)
Shape of generated embeddings =  (1, 67, 1024)
Shape of generated embeddings =  (1, 19, 1024)
Shape of generated embeddings =  (1, 77, 1024)
Shape of generated embeddings =  (1, 121, 1024)
Shape of generated embeddings =  (1, 6616, 1024)
Shape of generated embeddings =  (1, 110, 1024)
Shape of generated embeddings =  (1, 79, 1024)
Shape of generated embeddings =  (1, 41, 1024)
Shape of generated embeddings =  (1, 36, 1024)
Shape of generated embeddings =  (1, 47, 1024)
Shape of generated embeddings =  (1, 99, 1024)
Shape o

Shape of generated embeddings =  (1, 321, 1024)
Shape of generated embeddings =  (1, 34, 1024)
Shape of generated embeddings =  (1, 24, 1024)
Shape of generated embeddings =  (1, 166, 1024)
Shape of generated embeddings =  (1, 53, 1024)
Shape of generated embeddings =  (1, 15, 1024)
Shape of generated embeddings =  (1, 323, 1024)
Shape of generated embeddings =  (1, 109, 1024)
Shape of generated embeddings =  (1, 134, 1024)
Shape of generated embeddings =  (1, 252, 1024)
Shape of generated embeddings =  (1, 114, 1024)
Shape of generated embeddings =  (1, 63, 1024)
Shape of generated embeddings =  (1, 143, 1024)
Shape of generated embeddings =  (1, 94, 1024)
Shape of generated embeddings =  (1, 72, 1024)
Shape of generated embeddings =  (1, 73, 1024)
Shape of generated embeddings =  (1, 30, 1024)
Shape of generated embeddings =  (1, 64, 1024)
Shape of generated embeddings =  (1, 67, 1024)
Shape of generated embeddings =  (1, 382, 1024)
Shape of generated embeddings =  (1, 60, 1024)
Shap

Shape of generated embeddings =  (1, 117, 1024)
Shape of generated embeddings =  (1, 43, 1024)
Shape of generated embeddings =  (1, 58, 1024)
Shape of generated embeddings =  (1, 88, 1024)
Shape of generated embeddings =  (1, 32, 1024)
Shape of generated embeddings =  (1, 204, 1024)
Shape of generated embeddings =  (1, 127, 1024)
Shape of generated embeddings =  (1, 24, 1024)
Shape of generated embeddings =  (1, 342, 1024)
Shape of generated embeddings =  (1, 70, 1024)
Shape of generated embeddings =  (1, 50, 1024)
Shape of generated embeddings =  (1, 47, 1024)
Shape of generated embeddings =  (1, 207, 1024)
Shape of generated embeddings =  (1, 36, 1024)
Shape of generated embeddings =  (1, 52, 1024)
Shape of generated embeddings =  (1, 27, 1024)
Shape of generated embeddings =  (1, 58, 1024)
Shape of generated embeddings =  (1, 47, 1024)
Shape of generated embeddings =  (1, 241, 1024)
Shape of generated embeddings =  (1, 304, 1024)
Shape of generated embeddings =  (1, 339, 1024)
Shape

...

Shape of generated embeddings =  (1, 82, 1024)
Shape of generated embeddings =  (1, 135, 1024)
Shape of generated embeddings =  (1, 135, 1024)
Shape of generated embeddings =  (1, 169, 1024)
Shape of generated embeddings =  (1, 63, 1024)
Shape of generated embeddings =  (1, 93, 1024)
Shape of generated embeddings =  (1, 39, 1024)
Shape of generated embeddings =  (1, 114, 1024)
Shape of generated embeddings =  (1, 89, 1024)
Shape of generated embeddings =  (1, 155, 1024)
Shape of generated embeddings =  (1, 80, 1024)
Shape of generated embeddings =  (1, 74, 1024)
Shape of generated embeddings =  (1, 57, 1024)
Shape of generated embeddings =  (1, 40, 1024)
Shape of generated embeddings =  (1, 41, 1024)
Shape of generated embeddings =  (1, 110, 1024)
Shape of generated embeddings =  (1, 36, 1024)
Shape of generated embeddings =  (1, 68, 1024)
Shape of generated embeddings =  (1, 155, 1024)
Shape of generated embeddings =  (1, 16, 1024)
Shape of generated embeddings =  (1, 90, 1024)
Shape 

Shape of generated embeddings =  (1, 114, 1024)
Shape of generated embeddings =  (1, 1593, 1024)
Shape of generated embeddings =  (1, 85, 1024)
Shape of generated embeddings =  (1, 211, 1024)
Shape of generated embeddings =  (1, 145, 1024)
Shape of generated embeddings =  (1, 108, 1024)
Shape of generated embeddings =  (1, 94, 1024)
Shape of generated embeddings =  (1, 308, 1024)
Shape of generated embeddings =  (1, 181, 1024)
Shape of generated embeddings =  (1, 500, 1024)
Shape of generated embeddings =  (1, 43, 1024)
Shape of generated embeddings =  (1, 425, 1024)
Shape of generated embeddings =  (1, 58, 1024)
Shape of generated embeddings =  (1, 39, 1024)
Shape of generated embeddings =  (1, 99, 1024)
Shape of generated embeddings =  (1, 4, 1024)
Shape of generated embeddings =  (1, 39, 1024)
Shape of generated embeddings =  (1, 244, 1024)
Shape of generated embeddings =  (1, 291, 1024)
Shape of generated embeddings =  (1, 47, 1024)
Shape of generated embeddings =  (1, 56, 1024)
Sh

Shape of generated embeddings =  (1, 32, 1024)
Shape of generated embeddings =  (1, 50, 1024)
Shape of generated embeddings =  (1, 118, 1024)
Shape of generated embeddings =  (1, 39, 1024)
Shape of generated embeddings =  (1, 329, 1024)
Shape of generated embeddings =  (1, 97, 1024)
Shape of generated embeddings =  (1, 167, 1024)
Shape of generated embeddings =  (1, 176, 1024)
Shape of generated embeddings =  (1, 119, 1024)
Shape of generated embeddings =  (1, 144, 1024)
Shape of generated embeddings =  (1, 25, 1024)
Shape of generated embeddings =  (1, 332, 1024)
Shape of generated embeddings =  (1, 82, 1024)
Shape of generated embeddings =  (1, 69, 1024)
Shape of generated embeddings =  (1, 310, 1024)
Shape of generated embeddings =  (1, 151, 1024)
Shape of generated embeddings =  (1, 102, 1024)
Shape of generated embeddings =  (1, 5, 1024)
Shape of generated embeddings =  (1, 18, 1024)
Shape of generated embeddings =  (1, 58, 1024)
Shape of generated embeddings =  (1, 60, 1024)
Shap

Shape of generated embeddings =  (1, 111, 1024)
Shape of generated embeddings =  (1, 67, 1024)
Shape of generated embeddings =  (1, 646, 1024)
Shape of generated embeddings =  (1, 122, 1024)
Shape of generated embeddings =  (1, 423, 1024)
Shape of generated embeddings =  (1, 47, 1024)
Shape of generated embeddings =  (1, 82, 1024)
Shape of generated embeddings =  (1, 128, 1024)
Shape of generated embeddings =  (1, 374, 1024)
Shape of generated embeddings =  (1, 73, 1024)
Shape of generated embeddings =  (1, 98, 1024)
Shape of generated embeddings =  (1, 50, 1024)
Shape of generated embeddings =  (1, 53, 1024)
Shape of generated embeddings =  (1, 31, 1024)
Shape of generated embeddings =  (1, 70, 1024)
Shape of generated embeddings =  (1, 78, 1024)
Shape of generated embeddings =  (1, 42, 1024)
Shape of generated embeddings =  (1, 45, 1024)
Shape of generated embeddings =  (1, 33, 1024)
Shape of generated embeddings =  (1, 39, 1024)
Shape of generated embeddings =  (1, 61, 1024)
Shape o

Shape of generated embeddings =  (1, 116, 1024)
Shape of generated embeddings =  (1, 48, 1024)
Shape of generated embeddings =  (1, 39, 1024)
Shape of generated embeddings =  (1, 85, 1024)
Shape of generated embeddings =  (1, 37, 1024)
Shape of generated embeddings =  (1, 240, 1024)
Shape of generated embeddings =  (1, 102, 1024)
Shape of generated embeddings =  (1, 221, 1024)
Shape of generated embeddings =  (1, 101, 1024)
Shape of generated embeddings =  (1, 240, 1024)
Shape of generated embeddings =  (1, 120, 1024)
Shape of generated embeddings =  (1, 89, 1024)
Shape of generated embeddings =  (1, 168, 1024)
Shape of generated embeddings =  (1, 25, 1024)
Shape of generated embeddings =  (1, 94, 1024)
Shape of generated embeddings =  (1, 60, 1024)
Shape of generated embeddings =  (1, 68, 1024)
Shape of generated embeddings =  (1, 321, 1024)
Shape of generated embeddings =  (1, 128, 1024)
Shape of generated embeddings =  (1, 165, 1024)
Shape of generated embeddings =  (1, 26, 1024)
Sh

In [21]:
except_test_lst

[]